# Loading and splitting the data

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
# Load movie and ratings data
movies_df = pd.read_csv("~/Desktop/MovieLens-resources/movies.csv")

ratings_df = pd.read_csv("~/Desktop/MovieLens-resources/ratings.csv")
ratings_df = ratings_df.drop('timestamp',axis=1)

In [3]:
ratings_df = ratings_df.iloc[:100000]

In [4]:
ratings_df.shape

(100000, 3)

In [5]:
ratings_df

,userId,movieId,rating
0,1,1,4.0
1,1,110,4.0
2,1,158,4.0
3,1,260,4.5
4,1,356,5.0
...,...,...,...
99995,1035,66097,2.5
99996,1035,68157,5.0
99997,1035,68237,4.5
99998,1035,68954,2.5


In [6]:
# Splitting the data into train_data
train_data = ratings_df.iloc[:int(ratings_df.shape[0]*0.80)]
train_data.shape

(90000, 3)

In [7]:
# Splitting the data into test_data
test_data = ratings_df.iloc[int(ratings_df.shape[0]*0.80):]
test_data.shape

(10000, 3)

# SVD MF

http://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

__ Predicted Rating : __

𝑟̂ 𝑢𝑖=𝜇+𝑏𝑢+𝑏𝑖+𝑞𝑇𝑖𝑝𝑢 𝑞𝑞𝑖 - Representation of item(movie) in latent factor space

𝑝𝑝𝑢 - Representation of user in new latent factor space

A BASIC MATRIX FACTORIZATION MODEL in https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf Optimization problem with user item interactions and regularization (to avoid overfitting) ∑𝑟𝑢𝑖∈𝑅𝑡𝑟𝑎𝑖𝑛(𝑟𝑢𝑖−𝑟̂ 𝑢𝑖)2+𝜆(𝑏2𝑖+𝑏2𝑢+||𝑞𝑖||2+||𝑝𝑢||2)

In [8]:
from surprise import SVD
import numpy as np
import surprise
from surprise import Reader, Dataset
from surprise import accuracy


In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [10]:
# Standardize the 'rating' column
train_data['rating'] = scaler.fit_transform(train_data[['rating']])
test_data['rating'] = scaler.transform(test_data[['rating']])

/var/folders/hm/gjf74g9508ncjbckxvpd1lw00000gp/T/ipykernel_65637/493524336.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['rating'] = scaler.fit_transform(train_data[['rating']])
/var/folders/hm/gjf74g9508ncjbckxvpd1lw00000gp/T/ipykernel_65637/493524336.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['rating'] = scaler.transform(test_data[['rating']])


In [11]:
# Train set from train data
reader = Reader(rating_scale=(1,5))
    
data = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

In [12]:
svd_model = SVD()

trainset = data.build_full_trainset()
svd_model.fit(trainset)

In [13]:
testset = Dataset.load_from_df(test_data[['userId', 'movieId', 'rating']], reader)
testset = testset.build_full_trainset().build_testset()
predictions = svd_model.test(testset)

In [14]:
accuracy.rmse(predictions)

RMSE: 1.5572


1.5571816106498435